In [20]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import yfinance as yf

In [21]:
# Daten herunterladen und DataFrame erstellen
df = yf.download('TSLA', start='2015-01-01', end='2020-07-01')

# Index zurücksetzen und "Date" als separate Spalte erstellen
df.reset_index(inplace=True)

# Spalte "Date" umbenennen
df.rename(columns={'index': 'Date'}, inplace=True)

# Überprüfen Sie das aktualisierte DataFrame
print(df.head())

[*********************100%***********************]  1 of 1 completed
        Date       Open       High        Low      Close  Adj Close    Volume
0 2015-01-02  14.858000  14.883333  14.217333  14.620667  14.620667  71466000
1 2015-01-05  14.303333  14.433333  13.810667  14.006000  14.006000  80527500
2 2015-01-06  14.004000  14.280000  13.614000  14.085333  14.085333  93928500
3 2015-01-07  14.223333  14.318667  13.985333  14.063333  14.063333  44526000
4 2015-01-08  14.187333  14.253333  14.000667  14.041333  14.041333  51637500


In [22]:
print("Number of rows and columns:", df.shape)
df.head(5)

Number of rows and columns: (1383, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-01-02,14.858000,14.883333,14.217333,14.620667,14.620667,71466000
1,2015-01-05,14.303333,14.433333,13.810667,14.006000,14.006000,80527500
2,2015-01-06,14.004000,14.280000,13.614000,14.085333,14.085333,93928500
3,2015-01-07,14.223333,14.318667,13.985333,14.063333,14.063333,44526000
4,2015-01-08,14.187333,14.253333,14.000667,14.041333,14.041333,51637500


In [5]:
training_set = df.iloc[:800, 1:2].values
test_set = df.iloc[800:, 1:2].values

In [6]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, 800):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#(740, 60, 1)

SyntaxError: invalid syntax (1757222428.py, line 9)

In [7]:
sc = MinMaxScaler(feature_range=(0, 1))

training_set_scaled = sc.fit_transform(training_set)

X_train = []
y_train = []

for i in range(60, 800):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [8]:
model = Sequential()#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
24/24 [==============================] - 13s 81ms/step - loss: 0.0488
Epoch 2/100
24/24 [==============================] - 2s 79ms/step - loss: 0.0127
Epoch 3/100
24/24 [==============================] - 2s 79ms/step - loss: 0.0111
Epoch 4/100
24/24 [==============================] - 2s 81ms/step - loss: 0.0102
Epoch 5/100
24/24 [==============================] - 2s 84ms/step - loss: 0.0106
Epoch 6/100
24/24 [==============================] - 2s 86ms/step - loss: 0.0098
Epoch 7/100
24/24 [==============================] - 2s 89ms/step - loss: 0.0093
Epoch 8/100
24/24 [==============================] - 2s 86ms/step - loss: 0.0090
Epoch 9/100
24/24 [==============================] - 2s 87ms/step - loss: 0.0090
Epoch 10/100
24/24 [==============================] - 2s 89ms/step - loss: 0.0087
Epoch 11/100
24/24 [==============================] - 2s 86ms/step - loss: 0.0081
Epoch 12/100
24/24 [==============================] - 2s 87ms/step - loss: 0.0070
Epoch 13/100
24/24 [====

In [10]:
dataset_train = df.iloc[:800, 1:2]
dataset_test = df.iloc[800:, 1:2]

dataset_total = pd.concat((dataset_train, dataset_test), axis=0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1, 1)

inputs = sc.transform(inputs)

X_test = []

for i in range(60, 519):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(X_test.shape)


(459, 60, 1)


In [11]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

15/15 [==============================] - 4s 32ms/step


In [16]:
df.set_index("Date", inplace=True)  # Set "Date" column as the index

KeyError: "None of ['Date'] are in the columns"

In [14]:
# Visualising the results
plt.plot(df.loc[800:, "Date"],dataset_test.values, color = "red", label = "Real TESLA Stock Price")
plt.plot(df.loc[800:, "Date"],predicted_stock_price, color = "blue", label = "Predicted TESLA Stock Price")
plt.xticks(np.arange(0,459,50))
plt.title("TESLA Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("TESLA Stock Price")
plt.legend()
plt.show()

KeyError: 'Date'

In [19]:
import yfinance as yf
import pandas as pd

# Daten herunterladen und DataFrame erstellen
df2 = yf.download('TSLA', start='2015-01-01', end='2020-07-01')

# Index zurücksetzen und "Date" als separate Spalte erstellen
df2.reset_index(inplace=True)

# Spalte "Date" umbenennen
df2.rename(columns={'index': 'Date'}, inplace=True)

# Überprüfen Sie das aktualisierte DataFrame
print(df2.head())


[*********************100%***********************]  1 of 1 completed
        Date       Open       High        Low      Close  Adj Close    Volume
0 2015-01-02  14.858000  14.883333  14.217333  14.620667  14.620667  71466000
1 2015-01-05  14.303333  14.433333  13.810667  14.006000  14.006000  80527500
2 2015-01-06  14.004000  14.280000  13.614000  14.085333  14.085333  93928500
3 2015-01-07  14.223333  14.318667  13.985333  14.063333  14.063333  44526000
4 2015-01-08  14.187333  14.253333  14.000667  14.041333  14.041333  51637500
